import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
IMAGE_SIZE = (128, 128)
train_dir = r'C:\Users\User\Desktop\ARCHIVE\chest_xray\train'
test_dir = r'C:\Users\User\Desktop\ARCHIVE\chest_xray\test'

def load_images(directory):
    images = []
    labels = []
    for label in ['NORMAL', 'PNEUMONIA']:  # Classes
        label_path = os.path.normpath(os.path.join(directory, label))
        if not os.path.isdir(label_path):
            continue  # Skip if subdirectory doesn't exist
        for file in os.listdir(label_path):
            img_path = os.path.normpath(os.path.join(label_path, file))
            if file.endswith(('.png', '.jpg', '.jpeg')):
                try:
                    img = Image.open(img_path).convert('L')  # Convert to grayscale
                    img = img.resize(IMAGE_SIZE)  # Resize image
                    images.append(np.array(img) / 255.0)  # Normalize image
                    labels.append(0 if label == 'NORMAL' else 1)  # 0: normal, 1: pneumonia
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
    return np.array(images), np.array(labels)


train_images, train_labels = load_images(train_dir)
test_images, test_labels = load_images(test_dir)

In [ ]:
# Expand dimensions for CNN compatibility
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(train_images)

# Compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Learning rate schedule function
def lr_schedule(epoch, lr):
    if epoch > 20:
        return lr * 0.5
    elif epoch > 10:
        return lr * 0.75
    return lr

# Build the CNN model with L2 regularization and dropout
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (NORMAL vs PNEUMONIA)
])

# Compile the model using Adam optimizer with a learning rate schedule
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate scheduler and early stopping to avoid overfitting
lr_scheduler = LearningRateScheduler(lr_schedule)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Train the model with increased epochs
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    epochs=50,  # Increased number of epochs
                    validation_data=(test_images, test_labels),
                    class_weight=class_weights,
                    callbacks=[lr_scheduler, early_stopping])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Generate predictions and compute confusion matrix
y_pred = model.predict(test_images)
cm = confusion_matrix(test_labels, y_pred.round())

# Display the confusion matrix as a table
plt.figure(figsize=(6, 4))

# Create the table without using a heatmap
table_data = [['', 'Predicted: NORMAL', 'Predicted: PNEUMONIA'],
              ['Actual: NORMAL', cm[0, 0], cm[0, 1]],
              ['Actual: PNEUMONIA', cm[1, 0], cm[1, 1]]]

table = plt.table(cellText=table_data, cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(14)
table.scale(1.5, 1.5)  # Scale to make the table larger

# Remove axis for a clean look
plt.axis('off')
plt.title('Confusion Matrix', fontsize=16)
plt.show()